In [1]:
import torch
import torch.nn as nn

In [5]:
import torch
import numpy as np
from torchvision.transforms import Compose, ToTensor, Normalize

# 1. Khởi tạo ma trận ảnh 3x3 với mọi phần tử bằng 182 (kênh RGB)
image_np = np.full((3, 3, 3), 182, dtype=np.uint8)

# 2. Định nghĩa chuỗi biến đổi (Transform) theo gợi ý
transform = Compose([
    ToTensor(), # Chuyển pixel về [0, 1]
    Normalize(mean=[0.485, 0.456, 0.406], # Sử dụng Mean ImageNet
              std=[0.229, 0.224, 0.225])  # Sử dụng Std ImageNet
])

# 3. Thực hiện chuẩn hóa
# Lưu ý: transform kỳ vọng đầu vào là ảnh PIL hoặc mảng NumPy (H, W, C)
normalized_tensor = transform(image_np)
print(normalized_tensor)

# 4. Trích xuất kênh Đỏ (kênh đầu tiên) và làm tròn đến 1 chữ số thập phân
red_channel = normalized_tensor[0]
result = torch.round(red_channel, decimals=1)

print("Kết quả ma trận sau chuẩn hóa:")
print(result)

tensor([[[0.9988, 0.9988, 0.9988],
         [0.9988, 0.9988, 0.9988],
         [0.9988, 0.9988, 0.9988]],

        [[1.1506, 1.1506, 1.1506],
         [1.1506, 1.1506, 1.1506],
         [1.1506, 1.1506, 1.1506]],

        [[1.3677, 1.3677, 1.3677],
         [1.3677, 1.3677, 1.3677],
         [1.3677, 1.3677, 1.3677]]])
Kết quả ma trận sau chuẩn hóa:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [4]:
input_data = normalized_tensor[0:1, :, :] # Kích thước (1, 3, 3)
print(input_data)

tensor([[[0.9988, 0.9988, 0.9988],
         [0.9988, 0.9988, 0.9988],
         [0.9988, 0.9988, 0.9988]]])


In [6]:
kernel1 = torch.tensor([[[[1., 0.], 
                          [0., 1.]]]]) # Kernel 1
kernel2 = torch.tensor([[[[0., 1.], 
                          [1., 0.]]]]) # Kernel 2

# Thực hiện phép tính Convolution (Stride=1, Padding=0)
# Lưu ý: input_data cần có dạng (Batch, Channel, H, W) -> (1, 1, 3, 3)
input_4d = input_data.unsqueeze(0)

output_conv1 = torch.nn.functional.conv2d(input_4d, kernel1)
output_conv2 = torch.nn.functional.conv2d(input_4d, kernel2)

print("\n2. Kết quả sau lớp Convolution (làm tròn 1 chữ số):")
print(f"Nhánh trên (Kernel 1):\n {torch.round(output_conv1, decimals=1)}")
print(f"Nhánh dưới (Kernel 2):\n {torch.round(output_conv2, decimals=1)}")


2. Kết quả sau lớp Convolution (làm tròn 1 chữ số):
Nhánh trên (Kernel 1):
 tensor([[[[2., 2.],
          [2., 2.]]]])
Nhánh dưới (Kernel 2):
 tensor([[[[2., 2.],
          [2., 2.]]]])


In [8]:
relu_1 = torch.relu(output_conv1)
print(relu_1)
relu_2 = torch.relu(output_conv2)
print(relu_2)

tensor([[[[1.9976, 1.9976],
          [1.9976, 1.9976]]]])
tensor([[[[1.9976, 1.9976],
          [1.9976, 1.9976]]]])


In [9]:
import torch
import torch.nn as nn

# 1. Khởi tạo đầu vào (Kết quả từ lớp Convolution trước đó)
# Như đã tính toán, sau Convolution, cả 2 nhánh đều là ma trận 2x2 giá trị 2.0
# Kích thước: (Batch_size=1, Channels=1, Height=2, Width=2)
conv_output_branch1 = torch.full((1, 1, 2, 2), 2.0)
conv_output_branch2 = torch.full((1, 1, 2, 2), 2.0)

# 2. Định nghĩa hàm tính Batch Normalization thủ công theo sơ đồ
# Công thức: y = ((x - mean) / std) * gamma + beta
def manual_batch_norm(x, mean=0, std=1, gamma=0.5, beta=1):
    return ((x - mean) / std) * gamma + beta

# 3. Tính toán cho Nhánh 1 (Branch 1)
bn_out1 = manual_batch_norm(conv_output_branch1, mean=0, std=1, gamma=0.5, beta=1)
relu_out1 = torch.relu(bn_out1)

# 4. Tính toán cho Nhánh 2 (Branch 2)
bn_out2 = manual_batch_norm(conv_output_branch2, mean=0, std=1, gamma=0.5, beta=1)
relu_out2 = torch.relu(bn_out2)

# In kết quả kiểm tra
print("Kết quả sau ReLU - Nhánh trên:")
print(torch.round(relu_out1, decimals=1))

print("\nKết quả sau ReLU - Nhánh dưới:")
print(torch.round(relu_out2, decimals=1))

Kết quả sau ReLU - Nhánh trên:
tensor([[[[2., 2.],
          [2., 2.]]]])

Kết quả sau ReLU - Nhánh dưới:
tensor([[[[2., 2.],
          [2., 2.]]]])


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision.transforms import Compose, ToTensor, Normalize

# --- 1. TIỀN XỬ LÝ (NORMALIZATION) ---
# Khởi tạo ảnh 3x3 với giá trị pixel 182
image_np = np.full((3, 3, 3), 182, dtype=np.uint8)

# Định nghĩa transform với thông số ImageNet (Đáp án b từ câu 1)
transform = Compose([
    ToTensor(), # 182 -> 182/255 ≈ 0.7137
    Normalize(mean=[0.485, 0.456, 0.406], 
              std=[0.229, 0.224, 0.225])
])

# Ma trận đầu vào sau chuẩn hóa (x' ≈ 1.0)
normalized_input = transform(image_np)[0:1, :, :].unsqueeze(0) # (1, 1, 3, 3)
print(normalized_input)

# --- 2. ĐỊNH NGHĨA MÔ HÌNH CNN THEO SƠ ĐỒ ---
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolution layers với Kernel định nghĩa sẵn
        self.conv1 = nn.Conv2d(1, 1, kernel_size=2, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(1, 1, kernel_size=2, stride=1, padding=0, bias=False)
        
        # Gán trọng số thủ công cho Kernel 1 và Kernel 2
        with torch.no_grad():
            self.conv1.weight = nn.Parameter(torch.tensor([[[[1., 0.], [0., 1.]]]]))
            self.conv2.weight = nn.Parameter(torch.tensor([[[[0., 1.], [1., 0.]]]]))

        # Batch Normalization (tham số gamma=0.5, beta=1, mean=0, std=1)
        # Sử dụng tham số cố định thay vì học (track_running_stats=False)
        self.bn = nn.BatchNorm2d(1, affine=True, track_running_stats=False)
        with torch.no_grad():
            self.bn.weight.fill_(0.5) # gamma
            self.bn.bias.fill_(1.0)   # beta

        # Max Pooling
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected (Trọng số giả định là 1 và bias=1 theo sơ đồ)
        self.fc = nn.Linear(2, 1) # 2 đầu vào từ 2 nhánh, 1 đầu ra
        with torch.no_grad():
            self.fc.weight.fill_(1.0)
            self.fc.bias.fill_(1.0)

    def forward(self, x):
        # Nhánh 1
        x1 = self.conv1(x)
        x1 = self.bn(x1)
        x1 = F.relu(x1)
        
        # Nhánh 2
        x2 = self.conv2(x)
        x2 = self.bn(x2)
        x2 = F.relu(x2)
        
        # Max Pooling (Thực hiện trên từng nhánh sau đó gộp lại)
        p1 = self.maxpool(x1) # Trả về 1 giá trị
        p2 = self.maxpool(x2) # Trả về 1 giá trị
        
        # Flatten và đưa vào Fully Connected
        combined = torch.cat((p1.view(-1), p2.view(-1))).unsqueeze(0)
        out = self.fc(combined)
        return out

# --- 3. CHẠY MÔ HÌNH VÀ KIỂM TRA KẾT QUẢ ---
model = SimpleCNN()
model.eval() # Chuyển sang chế độ dự báo

output = model(normalized_input)

print(f"Giá trị đầu ra cuối cùng (Output '?'): {output.item():.1f}")


tensor([[[[0.9988, 0.9988, 0.9988],
          [0.9988, 0.9988, 0.9988],
          [0.9988, 0.9988, 0.9988]]]])
Giá trị đầu ra cuối cùng (Output '?'): 3.0


In [12]:
import torch
import torch.nn.functional as F
import numpy as np

# --- 1. CHUẨN HÓA ĐẦU VÀO (NORMALIZED INPUT) ---
# Pixel 182 -> ToTensor (182/255 = 0.7137) -> Normalize (mean=0.485, std=0.229)
pixel_val = 182 / 255.0
mean_red, std_red = 0.485, 0.229
x_prime = (pixel_val - mean_red) / std_red  # x' ≈ 1.0

# Tạo Tensor 1x3x3 đại diện cho "Normalized Input" trong sơ đồ
input_tensor = torch.full((1, 1, 3, 3), x_prime)
print(f"--- 0. ĐẦU VÀO SAU CHUẨN HÓA ---\n{input_tensor.round(decimals=1)}\n")

# --- 2. LỚP CONVOLUTION ---
# Định nghĩa 2 Kernel từ sơ đồ
kernel1 = torch.tensor([[[[1., 0.], [0., 1.]]]])
kernel2 = torch.tensor([[[[0., 1.], [1., 0.]]]])

conv_out1 = F.conv2d(input_tensor, kernel1, stride=1, padding=0)
conv_out2 = F.conv2d(input_tensor, kernel2, stride=1, padding=0)

print(f"--- 1. SAU CONVOLUTION ---")
print(f"Nhánh 1 (Kernel 1):\n{conv_out1.round(decimals=1)}")
print(f"Nhánh 2 (Kernel 2):\n{conv_out2.round(decimals=1)}\n")

# --- 3. LỚP BATCH NORM & RELU ---
def apply_bn_relu(x, gamma=0.5, beta=1.0, mean=0.0, std=1.0):
    # Công thức BatchNorm: y = ((x - mean) / std) * gamma + beta
    bn_out = ((x - mean) / std) * gamma + beta
    # Công thức ReLU: max(0, y)
    relu_out = torch.relu(bn_out)
    return relu_out

relu_out1 = apply_bn_relu(conv_out1)
relu_out2 = apply_bn_relu(conv_out2)

print(f"--- 2. SAU BATCH NORM & RELU ---")
print(f"Nhánh 1:\n{relu_out1.round(decimals=1)}")
print(f"Nhánh 2:\n{relu_out2.round(decimals=1)}\n")

# --- 4. LỚP MAX POOLING ---
# kernel_size=2, stride=2 sẽ nén ma trận 2x2 thành 1 giá trị lớn nhất
maxpool_out1 = F.max_pool2d(relu_out1, kernel_size=2, stride=2)
maxpool_out2 = F.max_pool2d(relu_out2, kernel_size=2, stride=2)

print(f"--- 3. SAU MAX POOLING (2x2 -> 1x1) ---")
print(f"Nhánh 1: {maxpool_out1.item():.1f}")
print(f"Nhánh 2: {maxpool_out2.item():.1f}\n")

# --- 5. LỚP FULLY CONNECTED (DỰ ĐOÁN ĐẦU RA CỐI CÙNG) ---
# Theo sơ đồ: Output = (Value1 * W1) + (Value2 * W2) + Bias
# Giả sử trọng số W=1 và Bias=1 (thông số phổ biến trong bài thi nếu không ghi rõ)
w1, w2, bias = 1.0, 1.0, 1.0
final_output = (maxpool_out1.item() * w1) + (maxpool_out2.item() * w2) + bias

print(f"--- 4. KẾT QUẢ CUỐI CÙNG (Dấu '?') ---")
print(f"Output = ({maxpool_out1.item():.1f} * {w1}) + ({maxpool_out2.item():.1f} * {w2}) + {bias}")
print(f"==> {final_output:.1f}")

--- 0. ĐẦU VÀO SAU CHUẨN HÓA ---
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

--- 1. SAU CONVOLUTION ---
Nhánh 1 (Kernel 1):
tensor([[[[2., 2.],
          [2., 2.]]]])
Nhánh 2 (Kernel 2):
tensor([[[[2., 2.],
          [2., 2.]]]])

--- 2. SAU BATCH NORM & RELU ---
Nhánh 1:
tensor([[[[2., 2.],
          [2., 2.]]]])
Nhánh 2:
tensor([[[[2., 2.],
          [2., 2.]]]])

--- 3. SAU MAX POOLING (2x2 -> 1x1) ---
Nhánh 1: 2.0
Nhánh 2: 2.0

--- 4. KẾT QUẢ CUỐI CÙNG (Dấu '?') ---
Output = (2.0 * 1.0) + (2.0 * 1.0) + 1.0
==> 5.0


In [18]:
import torch
import torch.nn as nn
from torchsummary import summary

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ConvBlock, self).__init__()
        # Nhánh chính (Main Path)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, 
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Nhánh tắt (Shortcut/Skip Connection)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual # Phép cộng Skip Connection
        out = self.relu(out)
        return out

class MyCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(MyCNN, self).__init__()
        # Initial Conv
        self.prep = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 32x32 -> 16x16
        )
        
        # Conv Block 1 & 2
        self.layer1 = ConvBlock(64, 64)   # Giữ nguyên 16x16
        self.pool1 = nn.MaxPool2d(2)      # 16x16 -> 8x8
        
        self.layer2 = ConvBlock(64, 128)  # Tăng channel lên 128, size 8x8
        self.pool2 = nn.MaxPool2d(2)      # 8x8 -> 4x4
        
        # Dense Block
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.prep(x)
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Khởi tạo mô hình và in summary
model = MyCNN()
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
        ConvBlock-11           [-1, 64, 16, 16]               0
        MaxPool2d-12             [-1, 64, 8, 8]               0
           Conv2d-13            [-1, 128, 8, 8]           8,192
      BatchNorm2d-14            [-1, 12

In [20]:
import torch
import torch.nn as nn
from torchsummary import summary

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Giả sử các giá trị thường gặp trong đáp án:
HIDDEN_NEURONS = 512 # Thử thay bằng 256 hoặc 1024
FINAL_MAP_SIZE = 4   # Thử thay bằng 4 nếu có thêm MaxPool

# Tính toán nhanh cho lớp Dense (phần chiếm nhiều tham số nhất)
input_nodes = 128 * FINAL_MAP_SIZE * FINAL_MAP_SIZE
dense_params = (input_nodes * HIDDEN_NEURONS) + HIDDEN_NEURONS
print(f"Tham số ước tính cho lớp Dense: {dense_params:,}")

Tham số ước tính cho lớp Dense: 1,049,088
